# Street canyon example in Radiosity

This example shows the process of simulating a geometry such as a street canyon while applying different properties to the wall boundaries. The setup of the canyon is comparable to ``Sound propagation in street canyons: Comparison between
diffusely and geometrically reflecting boundaries`` by Kang published in 2000.

Paper: https://doi.org/10.1121/1.428580


First, import all of our dependencies.

In [1]:
import numpy as np
import pyfar as pf
import sparrowpy as sp
import matplotlib.pyplot as plt
import os

%matplotlib inline
# %matplotlib ipympl

Define the street canyon and set general parameters for the simulation.

In [2]:
# canyon boundary dimensions in meters 120 20 18
length = 35  # X
width = 10  # Y
height = 4  # Z  (different heights were used by Kang)

# simulation parameters
patch_size = 4
etc_duration = 1
etc_time_resolution = 1 / 1000
max_reflection_order = 50
speed_of_sound = 343.2
absorption = 0.1

# create geometry for the two facades and the ground
boundaries = [
    sp.geometry.Polygon(
        np.array(
            [
                [0, 0, 0],
                [0, 0, height],
                [length, 0, 0],
                [length, 0, height],
            ],
        ),
        np.array([1, 0, 0]),  # up vector
        np.array([0, 1, 0]),  # normal vector
    ),
    sp.geometry.Polygon(
        np.array(
            [
                [0, 0, 0],
                [0, width, 0],
                [length, 0, 0],
                [length, width, 0],
            ],
        ),
        np.array([1, 0, 0]),
        np.array([0, 0, 1]),
    ),
    sp.geometry.Polygon(
        np.array(
            [
                [0, width, 0],
                [length, width, 0],
                [0, width, height],
                [length, width, height],
            ],
        ),
        np.array([1, 0, 0]),
        np.array([0, -1, 0]),
    ),
]

# set the same position for source and receiver
source = pf.Coordinates(30, 6, 1)
receiver = pf.Coordinates(30, 6, 1)

For further steps create the instance of the simulation class called ``DirectionalRadiosityFast`` from the previously defined boundaries and the patch size.
Two instances are created so that other properties can be compared later.

In [3]:
radiosity_diffuse = sp.DirectionalRadiosityFast.from_polygon(
    boundaries,
    patch_size,
)
radiosity_retroreflection = sp.DirectionalRadiosityFast.from_polygon(
    boundaries,
    patch_size,
)

In the diffuse case, the scattering coefficient is set to 1 for each boundary.

For the retroreflection, the provided BRDF data is used.

In [4]:
# create directional scattering data (totally diffuse) for one frequency band
brdf_sources = pf.Coordinates(0, 0, 1, weights=1)
brdf_receivers = pf.Coordinates(0, 0, 1, weights=1)
frequencies = np.array([8000])
brdf_diffuse = sp.brdf.create_from_scattering(
    brdf_sources,
    brdf_receivers,
    pf.FrequencyData(1, frequencies),  # diffuse scattering => 1
    pf.FrequencyData(absorption, frequencies),
)

# set the created scattering data in the radiosity instance
radiosity_diffuse.set_wall_brdf(
    np.arange(len(boundaries)).tolist(),
    brdf_diffuse,
    brdf_sources,
    brdf_receivers,
)


# use retroreflective scattering for the second simulation
brdf_rr, directions_bsc, directions_bsc = pf.io.read_sofa(
    os.path.join(os.curdir, "brdf_walls_RISC.sofa"),
)
radiosity_retroreflection.set_wall_brdf(
    np.arange(len(boundaries)).tolist(),
    brdf_rr,
    directions_bsc,
    directions_bsc,
)

SOFA file contained custom entries
----------------------------------
SourceWeights, ReceiverWeights


The air attenuation can be defined as well. Here, it is set to 0.

In [5]:
radiosity_diffuse.set_air_attenuation(
    pf.FrequencyData(
        np.zeros_like(brdf_diffuse.frequencies),
        brdf_diffuse.frequencies,
    ),
)

radiosity_retroreflection.set_air_attenuation(
    pf.FrequencyData(np.zeros_like(brdf_rr.frequencies), brdf_rr.frequencies),
)

Now the simulation start, first the geometry is baked, where all patch to patch relationships are precalculated. E.g. Form-factor including the geometrical relationship between the patches as well as the BRDFs. This is the most heavy part of the simulation and is independent of the source and receiver position.

In [6]:
radiosity_diffuse.bake_geometry()
radiosity_retroreflection.bake_geometry()

/home/ubunti/HiWi/sparrowpy/sparrowpy/form_factor/universal.py:48: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'C', False, aligned=True))
  form_factors[i,j] = universal_form_factor(


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(array(float64, 4d, C), Tuple(int64, int64, slice<a:b>, slice<a:b>), array(complex128, 2d, C))
 
There are 16 candidate implementations:
  - Of which 16 did not match due to:
  Overload of function 'setitem': File: <numerous>: Line N/A.
    With argument(s): '(array(float64, 4d, C), Tuple(int64, int64, slice<a:b>, slice<a:b>), array(complex128, 2d, C))':
   No match.

During: typing of setitem at /home/ubunti/HiWi/sparrowpy/sparrowpy/classes/RadiosityFast.py (1348)

File "../sparrowpy/classes/RadiosityFast.py", line 1348:
def _form_factors_with_directivity_dim(
    <source elided>
                    scattering, scattering_index)
                form_factors_tilde[i, j, :, :] = form_factors_tilde[
                ^

During: Pass nopython_type_inference

The next step is to calculate the initial energy for each patch.

In [ ]:
radiosity_diffuse.init_source_energy(source)
radiosity_retroreflection.init_source_energy(source)

Next we calculate the energy exchange between the patches, then we can collect the energy at the receiver.

In [ ]:
radiosity_diffuse.calculate_energy_exchange(
    speed_of_sound=speed_of_sound,
    etc_time_resolution=etc_time_resolution,
    etc_duration=etc_duration,
    max_reflection_order=max_reflection_order,
)
radiosity_retroreflection.calculate_energy_exchange(
    speed_of_sound=speed_of_sound,
    etc_time_resolution=etc_time_resolution,
    etc_duration=etc_duration,
    max_reflection_order=max_reflection_order,
)

The energy is collected at the receiver, this is quite fast and can be done for as many receivers as required.

In [ ]:
etc_diffuse = radiosity_diffuse.collect_energy_receiver_mono(
    receivers=receiver,
)
etc_retroreflection = radiosity_retroreflection.collect_energy_receiver_mono(
    receivers=receiver,
)

Finally, the two simulations can be compared.

In [ ]:
plt.figure()
pf.plot.time(
    etc_diffuse,
    dB=True,
    log_prefix=10,
    label="diffuse",
    linestyle="--",
)
pf.plot.time(
    etc_retroreflection,
    dB=True,
    log_prefix=10,
    label="retroreflection",
)

plt.legend()
plt.show()

In [ ]:
%load_ext watermark
%watermark -v -m -iv